In [590]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [591]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# CUSTOM LAYERS

In [592]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    self.a = tf.Variable(initial_value=tf.zeros(shape=[14,2]), trainable=False, name= self.name + "_activation")
    
  def call(self, inputs):
    a1a1 = tf.reshape(inputs[0],shape=(1,1))   #1
    a1a2 = tf.reshape(inputs[1],shape=(1,1))   #2 
    a1a2a2 = tf.reshape(inputs[2],shape=(1,1)) #3
    a1a2a1 = tf.reshape(inputs[3],shape=(1,1)) #4
    a2a2 = tf.reshape(inputs[4],shape=(1,1))   #5
    a2a1 = tf.reshape(inputs[5],shape=(1,1))   #6
    a2a1a1 = tf.reshape(inputs[6],shape=(1,1)) #7
    a2a1a2 = tf.reshape(inputs[7],shape=(1,1)) #8

    to_hidden = []
    to_hidden.append((tf.concat([a1a2a2, a2a2], axis = 1)))   #["TrRi"] [3,5]
    to_hidden.append((tf.concat([a1a2a1, a2a1], axis = 1)))   #["RiTr"] [4,6] 
    to_hidden.append((tf.concat([a1a1, a2a1], axis = 1)))     #["Plu"]  [1,6] 
    to_hidden.append((tf.concat([a1a2, a1a2a2], axis = 1)))   #["TTra"] [2,3] 
    to_hidden.append((tf.concat([a2a1, a2a1a1], axis = 1)))   #["TTrb"] [6,7] 
    to_hidden.append((tf.concat([a1a2, a2a1], axis = 1)))     #["Tr"]   [2,6]
    to_hidden.append((tf.concat([a1a2a1, a2a1a2], axis = 1))) #["MeTr"] [4,8] 
    to_hidden.append((tf.concat([a1a2a1, a1a2], axis = 1)))   #["MoTr"] [4,2] 
    to_hidden.append((tf.concat([a1a1, a1a2a1], axis = 1)))   #["RiRi"] [1,4] 
    to_hidden.append((tf.concat([a1a1, a1a2], axis = 1)))     #["MoFo"] [1,2] 
    to_hidden.append((tf.concat([a1a2a2, a1a2a1], axis = 1))) #["MMof"] [3,4] 
    to_hidden.append((tf.concat([a1a1, a1a2a2], axis = 1)))   #["MoRi"] [1,3] 
    to_hidden.append((tf.concat([a1a2a2, a2a1a1], axis = 1))) #["MeRi"] [3,7] 
    to_hidden.append((tf.concat([a1a1, a2a2], axis = 1)))     #["Ri"]   [1,5] 

    output = tf.concat(to_hidden,axis = 0)
    self.a.assign(output)
    return self.a.value()

In [593]:
class Layer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(Layer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val
    if self.name in ["HiddenLayer"]:
      self.type = "Hidden"
    elif self.name in ["OutputLayer"]:
      self.type = "Output"

  def build(self, input_shape):
    shape = [int(input_shape[-1]),self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
    
    if self.type == "Hidden":
      net = tf.matmul(inputs, self.w)[:,0]
    elif self.type == "Output":
      net = tf.matmul(inputs, self.w)
      
    self.a.assign(1/(1 + tf.exp((net + self.b)/temperature)))
    return self.a.value()


### Testing foward pass

In [594]:
x = tf.ones(shape=[3,4])
x

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [595]:
tf.reshape(tf.reduce_sum(x,axis=1), shape = [1, x.shape[0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4., 4., 4.]], dtype=float32)>

In [596]:
#x = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0])
x = tf.constant([0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0])
x

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 1., 1., 1.], dtype=float32)>

In [597]:
input_layer = InputLayer(name = "InputLayer")
input_layer

In [598]:
res_input = input_layer(x)
res_input

<tf.Tensor: shape=(14, 2), dtype=float32, numpy=
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [1., 1.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

In [599]:
input_layer.a

<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32, numpy=
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [1., 1.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

In [600]:
hidden_layer = Layer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.0)
hidden_layer

In [601]:
res_hidden = hidden_layer(inputs = res_input, temperature = 1)
res_hidden

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.37754068, 0.37754068, 0.37754068, 0.5       , 0.26894143,
        0.37754068, 0.37754068, 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.37754068, 0.37754068]], dtype=float32)>

In [602]:
hidden_layer.w

<tf.Variable 'HiddenLayer/HiddenLayer_weights:0' shape=(2, 14) dtype=float32, numpy=
array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5]], dtype=float32)>

In [603]:
hidden_layer.a

<tf.Variable 'HiddenLayer/HiddenLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.37754068, 0.37754068, 0.37754068, 0.5       , 0.26894143,
        0.37754068, 0.37754068, 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.37754068, 0.37754068]], dtype=float32)>

In [604]:
output_layer = Layer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.0)
output_layer

In [605]:
res_output = output_layer(inputs = res_hidden, temperature = 1)
res_output

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.02965795, 0.02965795, 0.02965795, 0.02965795, 0.02965795,
        0.02965795, 0.02965795, 0.02965795, 0.02965795, 0.02965795,
        0.02965795, 0.02965795, 0.02965795, 0.02965795]], dtype=float32)>

In [606]:
output_layer.w

<tf.Variable 'OutputLayer/OutputLayer_weights:0' shape=(14, 14) dtype=float32, numpy=
array([[0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
     

In [607]:
output_layer.a

<tf.Variable 'OutputLayer/OutputLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.02965795, 0.02965795, 0.02965795, 0.02965795, 0.02965795,
        0.02965795, 0.02965795, 0.02965795, 0.02965795, 0.02965795,
        0.02965795, 0.02965795, 0.02965795, 0.02965795]], dtype=float32)>

In [608]:
error = tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
error

<tf.Tensor: shape=(), dtype=float32, numpy=2.2317433>

### Test back propagation

In [609]:
alpha = 1

In [610]:
delta_Ao = res_output * (res_hidden - res_output) * (1 - res_output) + output_layer.deltaA * (res_hidden - res_output)
delta_Ao

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.01001149, 0.01001149, 0.01001149, 0.01353567, 0.00688618,
        0.01001149, 0.01001149, 0.01353567, 0.01353567, 0.01353567,
        0.01353567, 0.01353567, 0.01001149, 0.01001149]], dtype=float32)>

In [611]:
res_hidden

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.37754068, 0.37754068, 0.37754068, 0.5       , 0.26894143,
        0.37754068, 0.37754068, 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.37754068, 0.37754068]], dtype=float32)>

In [612]:
tf.transpose(res_hidden)

<tf.Tensor: shape=(14, 1), dtype=float32, numpy=
array([[0.37754068],
       [0.37754068],
       [0.37754068],
       [0.5       ],
       [0.26894143],
       [0.37754068],
       [0.37754068],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.37754068],
       [0.37754068]], dtype=float32)>

In [613]:
delta_Ao

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.01001149, 0.01001149, 0.01001149, 0.01353567, 0.00688618,
        0.01001149, 0.01001149, 0.01353567, 0.01353567, 0.01353567,
        0.01353567, 0.01353567, 0.01001149, 0.01001149]], dtype=float32)>

In [614]:
delta_Wo = alpha * tf.matmul(tf.transpose(res_hidden),delta_Ao)   #dubbio scambiare righe e colonne
delta_Wo

<tf.Tensor: shape=(14, 14), dtype=float32, numpy=
array([[0.00377975, 0.00377975, 0.00377975, 0.00511027, 0.00259981,
        0.00377975, 0.00377975, 0.00511027, 0.00511027, 0.00511027,
        0.00511027, 0.00511027, 0.00377975, 0.00377975],
       [0.00377975, 0.00377975, 0.00377975, 0.00511027, 0.00259981,
        0.00377975, 0.00377975, 0.00511027, 0.00511027, 0.00511027,
        0.00511027, 0.00511027, 0.00377975, 0.00377975],
       [0.00377975, 0.00377975, 0.00377975, 0.00511027, 0.00259981,
        0.00377975, 0.00377975, 0.00511027, 0.00511027, 0.00511027,
        0.00511027, 0.00511027, 0.00377975, 0.00377975],
       [0.00500575, 0.00500575, 0.00500575, 0.00676783, 0.00344309,
        0.00500575, 0.00500575, 0.00676783, 0.00676783, 0.00676783,
        0.00676783, 0.00676783, 0.00500575, 0.00500575],
       [0.00269251, 0.00269251, 0.00269251, 0.0036403 , 0.00185198,
        0.00269251, 0.00269251, 0.0036403 , 0.0036403 , 0.0036403 ,
        0.0036403 , 0.0036403 , 0.00269251

In [615]:
output_layer.w.assign_add(delta_Wo)
output_layer.w

<tf.Variable 'OutputLayer/OutputLayer_weights:0' shape=(14, 14) dtype=float32, numpy=
array([[0.59377974, 0.59377974, 0.59377974, 0.59511024, 0.5925998 ,
        0.59377974, 0.59377974, 0.59511024, 0.59511024, 0.59511024,
        0.59511024, 0.59511024, 0.59377974, 0.59377974],
       [0.59377974, 0.59377974, 0.59377974, 0.59511024, 0.5925998 ,
        0.59377974, 0.59377974, 0.59511024, 0.59511024, 0.59511024,
        0.59511024, 0.59511024, 0.59377974, 0.59377974],
       [0.59377974, 0.59377974, 0.59377974, 0.59511024, 0.5925998 ,
        0.59377974, 0.59377974, 0.59511024, 0.59511024, 0.59511024,
        0.59511024, 0.59511024, 0.59377974, 0.59377974],
       [0.5950057 , 0.5950057 , 0.5950057 , 0.5967678 , 0.59344304,
        0.5950057 , 0.5950057 , 0.5967678 , 0.5967678 , 0.5967678 ,
        0.5967678 , 0.5967678 , 0.5950057 , 0.5950057 ],
       [0.5926925 , 0.5926925 , 0.5926925 , 0.59364027, 0.59185195,
        0.5926925 , 0.5926925 , 0.59364027, 0.59364027, 0.59364027,
      

In [616]:
delta_Ah = res_hidden * (1 - res_hidden) * (tf.matmul(delta_Ao , output_layer.w)) # dubbio scambiare righe per colonne
delta_Ah

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.02209424, 0.02209424, 0.02209424, 0.02356485, 0.01844243,
        0.02209424, 0.02209424, 0.02356485, 0.02356485, 0.02356485,
        0.02356485, 0.02356485, 0.02209424, 0.02209424]], dtype=float32)>

In [617]:
delta_Wh = tf.transpose(alpha * tf.transpose(delta_Ah) * res_input)
delta_Wh

<tf.Tensor: shape=(2, 14), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.        , 0.01844243,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.02209424, 0.02209424, 0.02209424, 0.        , 0.01844243,
        0.02209424, 0.02209424, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.02209424, 0.02209424]], dtype=float32)>

In [618]:
hidden_layer.w.assign_add(delta_Wh)
hidden_layer.w

<tf.Variable 'HiddenLayer/HiddenLayer_weights:0' shape=(2, 14) dtype=float32, numpy=
array([[0.5       , 0.5       , 0.5       , 0.5       , 0.51844245,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       ],
       [0.52209425, 0.52209425, 0.52209425, 0.5       , 0.51844245,
        0.52209425, 0.52209425, 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.52209425, 0.52209425]], dtype=float32)>

---

# CUSTOM MODEL

In [619]:
class MQSelfReflexiveNetwork(tf.keras.Model):
  def __init__(self, learning_rate):
    super(MQSelfReflexiveNetwork, self).__init__(name='MQSelfReflexiveNetwork')

    self.input_layer = InputLayer(name = "InputLayer")
    self.h = Layer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.2)
    self.o = Layer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.2)
    self.temperature = tf.Variable(initial_value=1.0,trainable=False, name = "Temperature")
    self.error = tf.Variable(initial_value=0.0,trainable=False, name = "Error")
    self.alpha = learning_rate

  def call(self, input):
    ### FOWARD PASS
    res_input = self.input_layer(input)
    res_hidden = self.h(res_input, self.temperature)
    res_output = self.o(res_hidden, self.temperature)
    return tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
  
  def train_step(self, input):
    ### CALL FOWARD PASS
    self.error.assign(self(input, training = True))
    print("Input layer activations")
    print(self.input_layer.a)

    ### BACK PROPAGATION
    # Compute temperature
    self.temperature.assign(1 - (1 / (1 + self.error) ))

    # Update outputs weights
    delta_Ao = self.o.a * (self.h.a - self.o.a) * (1 - self.o.a) + (self.o.deltaA * ((self.h.a - self.o.a)) )
    delta_Wo = self.alpha * tf.matmul(tf.transpose(self.h.a),delta_Ao)
    self.o.deltaA.assign(delta_Ao)
    self.o.w.assign_add(delta_Wo)

#tf.reshape(tf.reduce_sum(x,axis=1), shape = [1, x.shape[0]])
    ## Update hidden weights
    delta_Ah = self.h.a * (1 - self.h.a) * tf.reshape( ( tf.reduce_sum((tf.transpose(self.o.deltaA) *  self.o.w), axis= 1)), shape = [1, 14] )
    delta_Wh = tf.transpose(self.alpha * tf.transpose(delta_Ah) * self.input_layer.a)
    self.h.deltaA.assign(delta_Ah)
    self.h.w.assign_add(delta_Wh)

    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weights": tf.squeeze(self.h.w.value()),
        "Output weights": tf.squeeze(self.o.w.value())
        }

In [620]:
# Construct an instance of CustomModel
input = tf.constant([0.01, 0.00, 0.03, 0.224, 0.875, 0.266, 0.427, 0.168])
#input = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
model = MQSelfReflexiveNetwork(learning_rate=10)

In [621]:
model.compile()
model_history = model.fit(x = input, epochs=200)

Epoch 1/200
Input layer activations
<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32>
Input layer activations
<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32>
1/1 [==============================] - 0s 249ms/step - Error: 1.6578 - Temperature: 0.6238 - Hidden weights: 0.5681 - Output weights: 0.6336
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - Error: 1.3450 - Temperature: 0.5736 - Hidden weights: 0.5918 - Output weights: 0.6525
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - Error: 1.1659 - Temperature: 0.5383 - Hidden weights: 0.6067 - Output weights: 0.6633
Epoch 4/200
1/1 [==============================] - 0s 5ms/step - Error: 1.5962 - Temperature: 0.6148 - Hidden weights: 0.6120 - Output weights: 0.6699
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - Error: 1.2907 - Temperature: 0.5634 - Hidden weights: 0.6233 - Output weights: 0.6780
Epoch 6/200
1/1 [==============================] - 0s 3ms/s

In [622]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weights,Output weights
0,1.657840,0.623755,"[[0.6031804, 0.5413673, 0.5479333, 0.5, 0.6802...","[[0.63379776, 0.6264973, 0.6259154, 0.6363434,..."
1,1.344985,0.573558,"[[0.60868835, 0.5858942, 0.57332426, 0.5279099...","[[0.6676126, 0.6412147, 0.64973265, 0.6679367,..."
2,1.165860,0.538290,"[[0.61010337, 0.6024244, 0.60463345, 0.5657351...","[[0.6930622, 0.6564573, 0.6600961, 0.68614995,..."
3,1.596234,0.614827,"[[0.61010337, 0.6040134, 0.6108197, 0.57632494...","[[0.70258236, 0.66473275, 0.66606116, 0.693731..."
4,1.290681,0.563449,"[[0.62323034, 0.6296244, 0.6108197, 0.59762275...","[[0.7155466, 0.6721992, 0.6793086, 0.70328337,..."
...,...,...,...,...
195,0.000000,0.000000,"[[0.8592487, 0.8932492, 0.86907536, 0.7675367,...","[[0.8666841, 0.81927496, 0.828295, 0.840817, 0..."
196,0.000000,0.000000,"[[0.8592487, 0.8932492, 0.86907536, 0.7675367,...","[[0.8666841, 0.81927496, 0.828295, 0.840817, 0..."
197,0.000000,0.000000,"[[0.8592487, 0.8932492, 0.86907536, 0.7675367,...","[[0.8666841, 0.81927496, 0.828295, 0.840817, 0..."
198,0.000000,0.000000,"[[0.8592487, 0.8932492, 0.86907536, 0.7675367,...","[[0.8666841, 0.81927496, 0.828295, 0.840817, 0..."


In [623]:
model.get_weights()[1][0]

array([0.8592487 , 0.8932492 , 0.86907536, 0.7675367 , 0.98429793,
       0.77939373, 0.9149864 , 0.8982889 , 0.84888494, 0.8679098 ,
       0.8049648 , 0.8769964 , 0.86114264, 0.9195248 ], dtype=float32)

In [624]:
model.get_weights()[1][1]

array([0.73225176, 0.9163267 , 0.92907405, 0.8708605 , 0.6993727 ,
       0.9823044 , 0.7190175 , 0.74858487, 0.90456504, 0.7430311 ,
       0.90435845, 0.8533661 , 0.6810275 , 0.7304508 ], dtype=float32)

In [625]:
dvf = abs(model.get_weights()[1][0] - model.get_weights()[1][1])
dvf

array([0.12699693, 0.02307749, 0.05999869, 0.10332382, 0.28492522,
       0.20291066, 0.19596887, 0.14970404, 0.0556801 , 0.1248787 ,
       0.09939367, 0.02363032, 0.18011516, 0.18907398], dtype=float32)

In [626]:
model.get_weights()[0]

array([[0.427, 0.03 ],
       [0.224, 0.   ],
       [0.168, 0.   ],
       [0.875, 0.427],
       [0.   , 0.01 ],
       [0.875, 0.   ],
       [0.224, 0.266],
       [0.224, 0.875],
       [0.168, 0.224],
       [0.168, 0.875],
       [0.427, 0.224],
       [0.168, 0.427],
       [0.427, 0.01 ],
       [0.168, 0.03 ]], dtype=float32)

In [627]:
model.get_weights()[0][:,0]

array([0.427, 0.224, 0.168, 0.875, 0.   , 0.875, 0.224, 0.224, 0.168,
       0.168, 0.427, 0.168, 0.427, 0.168], dtype=float32)

In [628]:
model.get_weights()[0][:,1]

array([0.03 , 0.   , 0.   , 0.427, 0.01 , 0.   , 0.266, 0.875, 0.224,
       0.875, 0.224, 0.427, 0.01 , 0.03 ], dtype=float32)

In [629]:
dva = abs(model.get_weights()[0][:,0] - model.get_weights()[0][:,1])
dva

array([0.39699998, 0.224     , 0.168     , 0.448     , 0.01      ,
       0.875     , 0.042     , 0.651     , 0.05600001, 0.707     ,
       0.20299998, 0.259     , 0.417     , 0.138     ], dtype=float32)

In [630]:
dvf

array([0.12699693, 0.02307749, 0.05999869, 0.10332382, 0.28492522,
       0.20291066, 0.19596887, 0.14970404, 0.0556801 , 0.1248787 ,
       0.09939367, 0.02363032, 0.18011516, 0.18907398], dtype=float32)

In [631]:
fig = px.line(df_model_history[["Error","Temperature"]])
fig.update_layout(title="Monitored variables",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
fig.show()

---